# Podstawy Przetwarzania Dużych Zbiorów Danych

Grupa IBD4 Poniedziałki 8:30

Łukasz Moskwa 247739 

Artur Szewczykowski 252546



# Część 1
 
1. https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-by-sex.md
2. https://medium.com/artificial-intelligence-101/data-analysis-with-google-bigquery-exploring-the-covid-19-open-data-public-dataset-2d246c66422d

# Część 2

In [77]:
# 2.5 Uruchom klienta BigQuery z poziomu Pythona.

import os

import pandas as pd
from google.cloud import bigquery

In [78]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="infinite-blade-452909-r4-d980a9c88b70.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client() 


In [79]:
# 2.6 Pobierz dane do obiektu DataFrame.

query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)    
query_result = query_job.result()  
df = query_result.to_dataframe()
df


C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AR,2021-07-13,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
1,BR,2020-04-20,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
2,BR,2021-01-25,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
3,BR,2020-04-09,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
4,BR,2021-07-30,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
5,BR,2020-06-07,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
6,BR,2020-05-14,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
7,BR,2020-01-19,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
8,BR,2021-05-06,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
9,BR,2021-07-26,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)


# Część 3
Misja dodatkowa - Poznanie danych

### 3.1 Ile jest zapisanych wierszy z danymi.

In [80]:
dataset_ref = client.dataset('covid19_open_data', project='bigquery-public-data')
table_ref = dataset_ref.table('covid19_open_data')
table = client.get_table(table_ref)

rows = table.num_rows

rows

22756333

### 3.2 Ile krajów jest uwzględnionych w danych.

In [81]:
query = ("""
SELECT COUNT(DISTINCT country_code) AS num_countries
FROM bigquery-public-data.covid19_open_data.covid19_open_data

""")

query_job = client.query(query)
result = query_job.result()

number_of_country = result.to_dataframe()

# Zwraca 246, ponieważ niektóre tereny podległe takie jak "French Southern Territories", "Guam" itd.
# Co jest niby okey ale nie są to państwa bo ich jest 195
number_of_country.iloc[0, 0]


C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


np.int64(246)

 ### 3.3. W jaki sposób zapisywane są dzienne informacje dla krajów.

In [82]:
# Dodają najczęsciej raz dziennie ale zdarza się, że kraj ma 2 wpisy ponieważ robi to dla innego aggregation_level
query = ("""
SELECT *
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE  aggregation_level = 0 AND country_name = 'Poland'
ORDER BY date ASC 
""")

country_all = client.query(query).to_dataframe()
country_all['date']

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
986    2022-09-13
987    2022-09-14
988    2022-09-15
989    2022-09-16
990    2022-09-17
Name: date, Length: 991, dtype: dbdate

In [83]:

is_continuous = (country_all['date'].diff().dropna() == pd.Timedelta(days=1)).all()

is_continuous


np.True_

### 3.4. W jaki sposób zapisywane są wartości liczbowe.

In [84]:
country_all.dtypes.value_counts()

float64    441
Int64      217
object      42
dbdate       1
Name: count, dtype: int64

Przykłady kolumn, które powiny być typem liczbowym a są obiektami czyli ciągami znaków.

In [85]:
# Wszystkie age_bins{1-9}
display(country_all[['population_largest_city', 'population_clustered', 'human_capital_index', 'area_rural_sq_km', 'area_urban_sq_km', 'life_expectancy']])

,population_largest_city,population_clustered,human_capital_index,area_rural_sq_km,area_urban_sq_km,life_expectancy
0,1775933,1775933,0.747,277840,30501,77.7
1,1775933,1775933,0.747,277840,30501,77.7
2,1775933,1775933,0.747,277840,30501,77.7
3,1775933,1775933,0.747,277840,30501,77.7
4,1775933,1775933,0.747,277840,30501,77.7
...,...,...,...,...,...,...
986,1775933,1775933,0.747,277840,30501,77.7
987,1775933,1775933,0.747,277840,30501,77.7
988,1775933,1775933,0.747,277840,30501,77.7
989,1775933,1775933,0.747,277840,30501,77.7


In [86]:
# Zapisane w typie coś / 1000
display(country_all.filter(like='_per_'))

,gdp_per_capita_usd,nurses_per_1000,physicians_per_1000,hospital_beds_per_1000
0,15222,6.8926,2.3788,NaN
1,15222,6.8926,2.3788,NaN
2,15222,6.8926,2.3788,NaN
3,15222,6.8926,2.3788,NaN
4,15222,6.8926,2.3788,NaN
...,...,...,...,...
986,15222,6.8926,2.3788,NaN
987,15222,6.8926,2.3788,NaN
988,15222,6.8926,2.3788,NaN
989,15222,6.8926,2.3788,NaN


###  3.5. Jaki przedział czasowy jest uwzględniony w danych. Jakie są przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób.

In [87]:
query_new_confirmed = ("""
SELECT date, SUM(new_confirmed)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_confirmed) IS NOT NULL AND SUM(new_confirmed) > 0
ORDER BY date ASC
""")

query_new_deceased = ("""
SELECT date, SUM(new_deceased)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_deceased) IS NOT NULL AND SUM(new_deceased) > 0
ORDER BY date ASC
""")

query_new_person_vaccinated = ("""
SELECT date, SUM(new_persons_vaccinated)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_persons_vaccinated) IS NOT NULL AND SUM(new_persons_vaccinated) > 0
ORDER BY date ASC
""")

new_confirmed = client.query(query_new_confirmed).to_dataframe()
new_deceased = client.query(query_new_deceased).to_dataframe()
new_person_vaccinated = client.query(query_new_person_vaccinated).to_dataframe()

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [88]:
print(f"Przedział czasowy danych jest od {country_all['date'].min()} do {country_all['date'].max()}.")
print(f"Przedział czasowy new_confirmed jest od {new_confirmed['date'].min()} do {new_confirmed['date'].max()}.")
print(f"Przedział czasowy new_deceased jest od {new_deceased['date'].min()} do {new_deceased['date'].max()}.")
print(f"Przedział czasowy new_person_vaccinated jest od {new_person_vaccinated['date'].min()} do {new_person_vaccinated['date'].max()}.")


Przedział czasowy danych jest od 2020-01-01 do 2022-09-17.
Przedział czasowy new_confirmed jest od 2020-01-01 do 2022-09-15.
Przedział czasowy new_deceased jest od 2020-01-02 do 2022-09-15.
Przedział czasowy new_person_vaccinated jest od 2020-12-07 do 2022-09-14.


In [89]:
# Dla krajów z osobna
query = """
SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_confirmed' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_confirmed IS NOT NULL AND new_confirmed > 0 AND aggregation_level=0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_deceased' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_deceased IS NOT NULL AND new_deceased > 0 AND aggregation_level=0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_persons_vaccinated' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_persons_vaccinated IS NOT NULL AND new_persons_vaccinated > 0 AND aggregation_level=0
GROUP BY country_name
ORDER BY country_name, metric
"""

dates_info = client.query(query).to_dataframe()
dates_info

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,country_name,start_date,end_date,metric
0,Afghanistan,2020-02-22,2022-09-13,new_confirmed
1,Afghanistan,2020-03-21,2022-09-12,new_deceased
2,Afghanistan,2021-02-28,2022-09-12,new_persons_vaccinated
3,Albania,2020-03-07,2022-09-13,new_confirmed
4,Albania,2020-03-09,2022-09-11,new_deceased
...,...,...,...,...
663,Zambia,2020-01-02,2022-09-10,new_deceased
664,Zambia,2021-04-15,2022-08-07,new_persons_vaccinated
665,Zimbabwe,2020-01-02,2022-09-12,new_confirmed
666,Zimbabwe,2020-01-02,2022-08-29,new_deceased


###  3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 


1. Ile kolumn liczy zbiór danych

In [90]:
num_columns = country_all.shape[1]
print(f"Liczba kolumn: {num_columns}")

Liczba kolumn: 701


2. Które państwa podawały dane z podziałem na płeć

In [91]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_male) AS confirmed_male, SUM(new_confirmed_female) AS confirmed_female, SUM(new_deceased_male) AS deceased_male, SUM(new_deceased_female) as deceased_female, SUM(new_tested_male) AS tested_male, SUM(new_tested_female) AS tested_female, SUM(new_hospitalized_patients_male) AS hospitalized_male, SUM(new_hospitalized_patients_female) AS hospitalized_female, SUM(new_intensive_care_patients_male) AS cared_male, SUM(new_intensive_care_patients_female) AS cared_female, SUM(new_recovered_male) AS recovered_male, SUM(new_recovered_female) AS recovered_female
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_male) IS NOT NULL OR SUM(new_deceased_male) IS NOT NULL OR SUM(new_tested_male) IS NOT NULL OR SUM(new_hospitalized_patients_male) IS NOT NULL OR SUM(new_recovered_male) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,confirmed_male,confirmed_female,deceased_male,deceased_female,tested_male,tested_female,hospitalized_male,hospitalized_female,cared_male,cared_female,recovered_male,recovered_female
0,Argentina,ARG,4418859,4780139,72801,53930,13178569,14928395,248464,222232,48422,33209,<NA>,<NA>
1,Brazil,BRA,8875904,8368673,311308,244105,26382353,26921275,996777,792931,330847,245941,0,0
2,Colombia,COL,2697358,3042821,93611,62159,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2542476,2898545
3,Czech Republic,CZE,1956993,2111418,23149,17783,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1925067,2082491
4,Estonia,EST,272596,326685,<NA>,<NA>,1595674,1890648,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,Finland,FIN,526714,587525,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,Germany,DEU,6092181,6356325,64006,56007,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4370388,4550358
7,Hong Kong,HKG,7417,<NA>,127,<NA>,<NA>,<NA>,6708,<NA>,<NA>,<NA>,<NA>,<NA>
8,India,IND,92449,51846,30,13,<NA>,<NA>,3431,1712,<NA>,<NA>,<NA>,<NA>
9,Malaysia,MYS,1676866,1323134,20855,15444,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


3. Które państwa podawały dane z podziałem na kategorie wiekowe

In [92]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_age_0) AS confirmed_age_0, SUM(new_deceased_age_0) AS deceased_age_0, SUM(new_tested_age_0) AS tested_age_0, SUM(new_hospitalized_patients_age_0) as hospitalized_age_0, SUM(new_intensive_care_patients_age_0) AS cared_age_0, SUM(new_recovered_age_0) AS recovered_age_0 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_age_0) IS NOT NULL OR SUM(new_deceased_age_0) IS NOT NULL OR SUM(new_tested_age_0) IS NOT NULL OR SUM(new_hospitalized_patients_age_0) IS NOT NULL OR SUM(new_recovered_age_0) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,confirmed_age_0,deceased_age_0,tested_age_0,hospitalized_age_0,cared_age_0,recovered_age_0
0,Argentina,ARG,224240,192,1303477,41095,3269,<NA>
1,Brazil,BRA,891485,1214,2726462,21195,5273,0
2,Colombia,COL,198102,360,<NA>,<NA>,<NA>,191387
3,Czech Republic,CZE,321365,7,<NA>,<NA>,<NA>,320168
4,Estonia,EST,44622,<NA>,208798,<NA>,<NA>,<NA>
5,Finland,FIN,85684,<NA>,<NA>,<NA>,<NA>,<NA>
6,France,FRA,2789829,<NA>,23039639,<NA>,<NA>,<NA>
7,Germany,DEU,437029,23,<NA>,<NA>,<NA>,290555
8,Hong Kong,HKG,666,0,<NA>,578,<NA>,<NA>
9,India,IND,4273,1,<NA>,78,<NA>,<NA>


4. Które państwa podawały dane dotyczące szczepień z podziałem na rodzaj(firmę) szczepionki

In [93]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_persons_fully_vaccinated_moderna) AS vaccinated_moderna, SUM(new_persons_fully_vaccinated_janssen) AS vaccinated_janssen, SUM(new_persons_fully_vaccinated_pfizer) AS vaccinated_pfizer, SUM(new_persons_vaccinated) AS total_vaccinated 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_persons_fully_vaccinated_moderna) IS NOT NULL OR SUM(new_persons_fully_vaccinated_janssen) IS NOT NULL OR SUM(new_persons_fully_vaccinated_pfizer) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,vaccinated_moderna,vaccinated_janssen,vaccinated_pfizer,total_vaccinated
0,Malaysia,MYS,<NA>,<NA>,15153452,27880068
1,United States of America,USA,63469471,16886719,112344792,263883327


5. Jakie jest znaczenie wartośći liczbowych z kolumn dotyczących restrykcji (school_closing, workplace_closing, cancel_public_events, restrictions on gatherings)

In [94]:
query = ("""
SELECT DISTINCT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(school_closing AS INT64) AS `school_closing`, CAST(workplace_closing  AS INT64) AS `workplace_closing`, CAST(cancel_public_events AS INT64) AS `cancel_public_events`, CAST(restrictions_on_gatherings AS INT64) AS `restrictions_on_gatherings`
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, school_closing, workplace_closing, cancel_public_events, restrictions_on_gatherings
ORDER BY date
""")
df_restrictions= client.query(query).to_dataframe()
df_restrictions

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2020-01-01,FSM,<NA>,<NA>,<NA>,<NA>
1,2020-01-01,BVT,<NA>,<NA>,<NA>,<NA>
2,2020-01-01,SMR,0,0,0,0
3,2020-01-01,KAZ,0,0,0,0
4,2020-01-01,MAC,0,0,0,0
...,...,...,...,...,...,...
243781,2022-09-17,KWT,<NA>,<NA>,<NA>,<NA>
243782,2022-09-17,CHN,<NA>,<NA>,<NA>,<NA>
243783,2022-09-17,LIE,<NA>,<NA>,<NA>,<NA>
243784,2022-09-17,DNK,<NA>,<NA>,<NA>,<NA>


<b>School closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub wprowadzenie obostrzeń,
<br> 2 - nakaz zamknięcia niektórych typów szkół,
<br> 3 - nakaz zamknięcia wszystkich rodzai szkół. <br> <br>
<b>Workplace closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub praca zdalna, albo wprowadzenie obostrzeń dla firm,
<br> 2 - nakaz zamknięcia niektórych sektorów lub obowiązkowa praca zdalna dla części pracowników,
<br> 3 - nakaz zamknięcia wszystkich miejsc pracy z wyjątkiem działalności niezbędnych (np. sklepy spożywcze. placówki medyczne). <br> <br>
<b>Cancel public events:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane unikanie/odwoływanie,
<br> 2 - zakaz organizacji. <br> <br>
<b>Restrictions on gatherings:</b>
<br> 0 - brak ograniczeń,
<br> 1 - ograniczenia dla bardzo dużych zgromadzeń (powyżej 1000 osób),
<br> 2 - ograniczenia dla zgromadzeń od 101 do 1000 osób,
<br> 3 - ograniczenia dla zgromadzeń od 11 do 100 osób,
<br> 4 - ograniczenia dla zgromadzeń do 10 osób lub mniej.
<br>
via https://github.com/OxCGRT/covid-policy-dataset/blob/main/documentation_and_codebook.md <br>

# Część 4
Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.


### 4.1 Podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [95]:
query = ("""
SELECT DISTINCT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY country_name, iso_3166_1_alpha_3, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
""")

country_data = client.query(query).to_dataframe()
country_data["GDP_2022"] = country_data["population"] * country_data["gdp_per_capita_usd"]
country_data

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd,GDP_2022
0,Romania,ROU,19414458,238400,83.100,10468793,12390,240545134620
1,Armenia,ARM,2963234,29740,104.083,1869848,4622,13696067548
2,Burkina Faso,BFA,20903278,274220,76.401,6092349,774,16179137172
3,Belarus,BLR,9449321,207600,46.569,7482982,6663,62960825823
4,Costa Rica,CRI,5094114,51100,99.767,4041885,12238,62341767132
...,...,...,...,...,...,...,...,...
241,Slovenia,SVN,2080908,20675,102.900,1144654,26078,54265918824
242,Somalia,SOM,15893219,637660,25.334,7034861,<NA>,<NA>
243,El Salvador,SLV,6486201,21040,313.041,4694702,4187,27157723587
244,Turkey,TUR,82003882,785350,106.200,63097818,9042,741479101044


### 4.2 statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [96]:
# Tutaj są wartości null w sumie. Można podmienić na 0 COALESCE(SUM(new_confirmed), 0) ale oznaczała by to, że nikt tam nie chorował a naprawdę nie ma danych.
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, new_confirmed as `Total_confirmed_cases`,
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_confirmed
ORDER BY date
""")

covid_statistic = client.query(query).to_dataframe()
covid_statistic.dropna(subset=['Total_confirmed_cases'], inplace=True)
covid_statistic

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_confirmed_cases
1,2020-01-01,AND,0
2,2020-01-01,LTU,0
3,2020-01-01,SOM,0
5,2020-01-01,BRA,380
8,2020-01-01,REU,0
...,...,...,...
243202,2022-09-15,IDN,2651
243234,2022-09-15,ITA,17978
243236,2022-09-15,CHN,46558
243269,2022-09-15,MYS,2375


### 4.3 Uwypuklenie problemu śmiertelności.

In [97]:
query = ("""
SELECT date,iso_3166_1_alpha_3 AS `3_letter_code`, new_deceased as `Total_new_deceased` 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_deceased
""")

covid_deceased = client.query(query).to_dataframe()
covid_deceased.dropna(subset=['Total_new_deceased'], inplace=True)
covid_deceased

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_new_deceased
0,2020-08-25,BRA,1271
1,2021-04-30,BRA,2595
2,2020-09-06,COL,238
3,2021-03-29,DEU,290
4,2021-02-20,DEU,96
...,...,...,...
243781,2021-01-07,PRY,15
243782,2020-12-18,KOR,15
243783,2021-07-09,EGY,15
243784,2020-03-30,ECU,15


### 4.4 Trendy i zależności dotyczące szczepień na COVID-19.

In [98]:
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`,
new_persons_vaccinated as `Total_new_vaccinations`,
new_persons_fully_vaccinated as`New_persons_fully_vaccinated`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_persons_vaccinated, new_persons_fully_vaccinated
""")

covid_trend_analysis = client.query(query).to_dataframe()
covid_trend_analysis.dropna(subset=['Total_new_vaccinations', 'New_persons_fully_vaccinated'], how='all', inplace=True)
covid_trend_analysis

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_new_vaccinations,New_persons_fully_vaccinated
0,2021-03-28,BRA,223463,23326
1,2022-01-19,MEX,50144,197473
2,2022-01-13,MYS,2244,3648
3,2021-07-26,NZL,20554,9983
4,2021-06-07,PER,7996,23562
...,...,...,...,...
243781,2021-01-09,BRA,<NA>,0
243782,2021-03-28,ALB,0,0
243783,2021-09-30,QAT,0,0
243784,2022-05-01,SYC,0,0


### 4.5 Liczba wykonanych testów

In [99]:
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(investment_in_vaccines AS INT64) as `Investment_in_vaccines`, CAST(new_tested AS INT64) as `Total_tested`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, investment_in_vaccines, new_tested
ORDER BY date
""")

investment_in_vaccines = client.query(query).to_dataframe()
investment_in_vaccines.dropna(subset=['Total_tested'], how='all', inplace=True)

display(investment_in_vaccines)

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Investment_in_vaccines,Total_tested
39,2020-01-01,MEX,0,72
109,2020-01-01,ARG,0,4
172,2020-01-01,BRA,0,1154
306,2020-01-02,MEX,0,89
419,2020-01-02,ARG,0,42
...,...,...,...,...
243078,2022-09-15,AUT,<NA>,67392
243116,2022-09-15,ITA,<NA>,146983
243136,2022-09-15,AUS,<NA>,4367
243231,2022-09-15,MYS,<NA>,0


In [100]:
df_restrictions

,date,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2020-01-01,FSM,<NA>,<NA>,<NA>,<NA>
1,2020-01-01,BVT,<NA>,<NA>,<NA>,<NA>
2,2020-01-01,SMR,0,0,0,0
3,2020-01-01,KAZ,0,0,0,0
4,2020-01-01,MAC,0,0,0,0
...,...,...,...,...,...,...
243781,2022-09-17,KWT,<NA>,<NA>,<NA>,<NA>
243782,2022-09-17,CHN,<NA>,<NA>,<NA>,<NA>
243783,2022-09-17,LIE,<NA>,<NA>,<NA>,<NA>
243784,2022-09-17,DNK,<NA>,<NA>,<NA>,<NA>


# Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [101]:
df_data_from_4 = pd.merge(country_data, covid_statistic, on=['3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, covid_deceased, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, covid_trend_analysis, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, investment_in_vaccines, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, df_restrictions, on =['date', '3_letter_code'])
df_data_from_4.to_csv("..\wyniki\\data_from_4.csv", index=False)
df_data_from_4

<>:6: SyntaxWarning: invalid escape sequence '\w'
<>:6: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_12068\644546437.py:6: SyntaxWarning: invalid escape sequence '\w'
  df_data_from_4.to_csv("..\wyniki\\data_from_4.csv", index=False)


,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,Investment_in_vaccines,Total_tested,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,Armenia,ARM,2963234,29740,104.083,1869848,4622,13696067548,2021-05-09,243,7,10162,<NA>,<NA>,2678,<NA>,<NA>,<NA>,<NA>
1,Armenia,ARM,2963234,29740,104.083,1869848,4622,13696067548,2021-05-16,212,13,8522,<NA>,<NA>,3771,<NA>,<NA>,<NA>,<NA>
2,Armenia,ARM,2963234,29740,104.083,1869848,4622,13696067548,2021-05-23,157,6,7313,<NA>,<NA>,3205,<NA>,<NA>,<NA>,<NA>
3,Armenia,ARM,2963234,29740,104.083,1869848,4622,13696067548,2021-05-30,108,7,6967,<NA>,<NA>,3280,<NA>,<NA>,<NA>,<NA>
4,Armenia,ARM,2963234,29740,104.083,1869848,4622,13696067548,2021-06-06,73,0,12974,<NA>,<NA>,3067,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Turkey,TUR,82003882,785350,106.200,63097818,9042,741479101044,2022-05-27,940,4,873,1150,0,134252,0,0,0,0
31436,Turkey,TUR,82003882,785350,106.200,63097818,9042,741479101044,2022-05-28,966,3,304,479,0,138752,0,0,0,0
31437,Turkey,TUR,82003882,785350,106.200,63097818,9042,741479101044,2022-05-29,864,2,144,279,0,129961,0,0,0,0
31438,Turkey,TUR,82003882,785350,106.200,63097818,9042,741479101044,2022-05-30,908,4,973,1489,0,133352,0,0,0,0


# Część 6


### 6.1 Usunięcie powtarzających się kolumn, usuwamy z naszego zbioru, ponieważ zbiory znalezione potem okazują się być dokładniejsze.

In [102]:
df_data_from_4.drop('population', axis=1, inplace=True)
df_data_from_4.drop('area_sq_km', axis=1, inplace=True)
df_data_from_4.drop('population_density', axis=1, inplace=True)

df_data_from_4

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,Investment_in_vaccines,Total_tested,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,Armenia,ARM,1869848,4622,13696067548,2021-05-09,243,7,10162,<NA>,<NA>,2678,<NA>,<NA>,<NA>,<NA>
1,Armenia,ARM,1869848,4622,13696067548,2021-05-16,212,13,8522,<NA>,<NA>,3771,<NA>,<NA>,<NA>,<NA>
2,Armenia,ARM,1869848,4622,13696067548,2021-05-23,157,6,7313,<NA>,<NA>,3205,<NA>,<NA>,<NA>,<NA>
3,Armenia,ARM,1869848,4622,13696067548,2021-05-30,108,7,6967,<NA>,<NA>,3280,<NA>,<NA>,<NA>,<NA>
4,Armenia,ARM,1869848,4622,13696067548,2021-06-06,73,0,12974,<NA>,<NA>,3067,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Turkey,TUR,63097818,9042,741479101044,2022-05-27,940,4,873,1150,0,134252,0,0,0,0
31436,Turkey,TUR,63097818,9042,741479101044,2022-05-28,966,3,304,479,0,138752,0,0,0,0
31437,Turkey,TUR,63097818,9042,741479101044,2022-05-29,864,2,144,279,0,129961,0,0,0,0
31438,Turkey,TUR,63097818,9042,741479101044,2022-05-30,908,4,973,1489,0,133352,0,0,0,0


### 6.2 Podstawowe infromacje geograficzne i demograficzne o krajach

In [107]:
# Tutaj możemy usunąć dane z 4.1, ponieważ są one już w pliku .csv z 4.1

world_countries = pd.read_csv("world_countries.csv")

df_with_countries = pd.merge(df_data_from_4, world_countries, left_on=['3_letter_code'], right_on=['CCA3'], how="left")
df_with_countries.drop('Country/Territory', axis=1, inplace=True)
df_with_countries.drop('CCA3', axis=1, inplace=True)
df_with_countries["2021 Population"] = (df_with_countries["2020 Population"] + df_with_countries["2022 Population"]) / 2
df_with_countries

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,2021 Population
0,Armenia,ARM,1869848,4622,13696067548,2021-05-09,243,7,10162,<NA>,...,2946293,3168523,3556539,3135123,2534377,29743,93.4831,0.9962,0.03,2793038.5
1,Armenia,ARM,1869848,4622,13696067548,2021-05-16,212,13,8522,<NA>,...,2946293,3168523,3556539,3135123,2534377,29743,93.4831,0.9962,0.03,2793038.5
2,Armenia,ARM,1869848,4622,13696067548,2021-05-23,157,6,7313,<NA>,...,2946293,3168523,3556539,3135123,2534377,29743,93.4831,0.9962,0.03,2793038.5
3,Armenia,ARM,1869848,4622,13696067548,2021-05-30,108,7,6967,<NA>,...,2946293,3168523,3556539,3135123,2534377,29743,93.4831,0.9962,0.03,2793038.5
4,Armenia,ARM,1869848,4622,13696067548,2021-06-06,73,0,12974,<NA>,...,2946293,3168523,3556539,3135123,2534377,29743,93.4831,0.9962,0.03,2793038.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Turkey,TUR,63097818,9042,741479101044,2022-05-27,940,4,873,1150,...,73195345,64113547,54324142,44089069,35540990,783562,108.9145,1.0067,1.07,84738334.5
31436,Turkey,TUR,63097818,9042,741479101044,2022-05-28,966,3,304,479,...,73195345,64113547,54324142,44089069,35540990,783562,108.9145,1.0067,1.07,84738334.5
31437,Turkey,TUR,63097818,9042,741479101044,2022-05-29,864,2,144,279,...,73195345,64113547,54324142,44089069,35540990,783562,108.9145,1.0067,1.07,84738334.5
31438,Turkey,TUR,63097818,9042,741479101044,2022-05-30,908,4,973,1489,...,73195345,64113547,54324142,44089069,35540990,783562,108.9145,1.0067,1.07,84738334.5


In [108]:
df_gdp = pd.read_csv("gdp.csv")
df_gdp_pivot = df_gdp.pivot(index="Country Code", columns="Year", values="Value")
df_gdp_pivot.columns = [f"GDP_{int(year)}" for year in df_gdp_pivot.columns]
df_gdp_pivot.reset_index(inplace=True)
df_with_gdp = df_with_countries.merge(df_gdp_pivot, left_on="3_letter_code", right_on="Country Code", how="left")
df_with_gdp.drop(columns=["Country Code"], inplace=True)
display(df_with_gdp)

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,GDP_2007,GDP_2008,GDP_2009,GDP_2010,GDP_2011,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016
0,Armenia,ARM,1869848,4622,13696067548,2021-05-09,243,7,10162,<NA>,...,9.206302e+09,1.166204e+10,8.647937e+09,9.260285e+09,1.014211e+10,1.061932e+10,1.112147e+10,1.160951e+10,1.055334e+10,1.057230e+10
1,Armenia,ARM,1869848,4622,13696067548,2021-05-16,212,13,8522,<NA>,...,9.206302e+09,1.166204e+10,8.647937e+09,9.260285e+09,1.014211e+10,1.061932e+10,1.112147e+10,1.160951e+10,1.055334e+10,1.057230e+10
2,Armenia,ARM,1869848,4622,13696067548,2021-05-23,157,6,7313,<NA>,...,9.206302e+09,1.166204e+10,8.647937e+09,9.260285e+09,1.014211e+10,1.061932e+10,1.112147e+10,1.160951e+10,1.055334e+10,1.057230e+10
3,Armenia,ARM,1869848,4622,13696067548,2021-05-30,108,7,6967,<NA>,...,9.206302e+09,1.166204e+10,8.647937e+09,9.260285e+09,1.014211e+10,1.061932e+10,1.112147e+10,1.160951e+10,1.055334e+10,1.057230e+10
4,Armenia,ARM,1869848,4622,13696067548,2021-06-06,73,0,12974,<NA>,...,9.206302e+09,1.166204e+10,8.647937e+09,9.260285e+09,1.014211e+10,1.061932e+10,1.112147e+10,1.160951e+10,1.055334e+10,1.057230e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Turkey,TUR,63097818,9042,741479101044,2022-05-27,940,4,873,1150,...,6.757701e+11,7.643357e+11,6.446399e+11,7.718768e+11,8.325463e+11,8.739818e+11,9.505953e+11,9.341678e+11,8.597942e+11,8.637117e+11
31436,Turkey,TUR,63097818,9042,741479101044,2022-05-28,966,3,304,479,...,6.757701e+11,7.643357e+11,6.446399e+11,7.718768e+11,8.325463e+11,8.739818e+11,9.505953e+11,9.341678e+11,8.597942e+11,8.637117e+11
31437,Turkey,TUR,63097818,9042,741479101044,2022-05-29,864,2,144,279,...,6.757701e+11,7.643357e+11,6.446399e+11,7.718768e+11,8.325463e+11,8.739818e+11,9.505953e+11,9.341678e+11,8.597942e+11,8.637117e+11
31438,Turkey,TUR,63097818,9042,741479101044,2022-05-30,908,4,973,1489,...,6.757701e+11,7.643357e+11,6.446399e+11,7.718768e+11,8.325463e+11,8.739818e+11,9.505953e+11,9.341678e+11,8.597942e+11,8.637117e+11


# Część 7

### 7.1 Dane związane z inflacją

In [109]:
# https://www.worldbank.org/en/research/brief/inflation-database

# Z wszystkich strony arkusza wybraliśmy ogólny przyrost inflacji w danych latach
df_inflation = pd.read_excel("load_data\\inflation_all.xlsx", sheet_name="hcpi_a")

df_inflation.drop(columns=["IMF Country Code", "Indicator Type", "Series Name", "Note", "Country"], inplace=True)
df_inflation = df_inflation.melt(id_vars=["Country Code"], var_name="Year", value_name="Inflation")
df_inflation.dropna(subset=["Inflation"], inplace=True)
df_inflation = df_inflation.pivot(index="Country Code", columns="Year", values="Inflation")
df_inflation.columns = [f"Inflation_{int(year)}" for year in df_inflation.columns]
df_inflation.reset_index(inplace=True)
df_with_inflation = df_with_gdp.merge(df_inflation, left_on="3_letter_code", right_on="Country Code", how="left")
df_with_inflation.drop(columns=["Country Code"], inplace=True)
display(df_with_inflation)

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,Inflation_2014,Inflation_2015,Inflation_2016,Inflation_2017,Inflation_2018,Inflation_2019,Inflation_2020,Inflation_2021,Inflation_2022,Inflation_2023
0,Armenia,ARM,1869848,4622,13696067548,2021-05-09,243,7,10162,<NA>,...,2.981000,3.73100,-1.406000,1.031,2.52,1.443,1.211436,7.184836,8.640911,1.980419
1,Armenia,ARM,1869848,4622,13696067548,2021-05-16,212,13,8522,<NA>,...,2.981000,3.73100,-1.406000,1.031,2.52,1.443,1.211436,7.184836,8.640911,1.980419
2,Armenia,ARM,1869848,4622,13696067548,2021-05-23,157,6,7313,<NA>,...,2.981000,3.73100,-1.406000,1.031,2.52,1.443,1.211436,7.184836,8.640911,1.980419
3,Armenia,ARM,1869848,4622,13696067548,2021-05-30,108,7,6967,<NA>,...,2.981000,3.73100,-1.406000,1.031,2.52,1.443,1.211436,7.184836,8.640911,1.980419
4,Armenia,ARM,1869848,4622,13696067548,2021-06-06,73,0,12974,<NA>,...,2.981000,3.73100,-1.406000,1.031,2.52,1.443,1.211436,7.184836,8.640911,1.980419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Turkey,TUR,63097818,9042,741479101044,2022-05-27,940,4,873,1150,...,8.854579,7.67085,7.775131,11.144,16.33,15.177,12.279000,19.596490,72.308840,53.859409
31436,Turkey,TUR,63097818,9042,741479101044,2022-05-28,966,3,304,479,...,8.854579,7.67085,7.775131,11.144,16.33,15.177,12.279000,19.596490,72.308840,53.859409
31437,Turkey,TUR,63097818,9042,741479101044,2022-05-29,864,2,144,279,...,8.854579,7.67085,7.775131,11.144,16.33,15.177,12.279000,19.596490,72.308840,53.859409
31438,Turkey,TUR,63097818,9042,741479101044,2022-05-30,908,4,973,1489,...,8.854579,7.67085,7.775131,11.144,16.33,15.177,12.279000,19.596490,72.308840,53.859409


### Dane o bezrobociu

In [110]:
#  https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS?view=map&year=1991

df_unemployment = pd.read_csv("load_data\\total_unemployment.csv")
df_unemployment.drop(columns=["Country Name", "Indicator Name", "Indicator Code", "Unnamed: 68"], inplace=True, errors="ignore")
df_unemployment = df_unemployment.melt(id_vars=["Country Code"], var_name="Year", value_name="Unemployment")
df_unemployment.dropna(subset=["Unemployment"], inplace=True)
df_unemployment = df_unemployment.pivot(index="Country Code", columns="Year", values="Unemployment")
df_unemployment.columns = [f"Unemployment_{int(year)}" for year in df_unemployment.columns]
df_unemployment.reset_index(inplace=True)
df_unemployment = df_with_inflation.merge(df_unemployment, left_on="3_letter_code", right_on="Country Code", how="left")
df_unemployment.drop(columns=["Country Code"], inplace=True)
display(df_unemployment)

,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,Unemployment_2014,Unemployment_2015,Unemployment_2016,Unemployment_2017,Unemployment_2018,Unemployment_2019,Unemployment_2020,Unemployment_2021,Unemployment_2022,Unemployment_2023
0,Armenia,ARM,1869848,4622,13696067548,2021-05-09,243,7,10162,<NA>,...,17.498,18.261,17.617,17.704,18.966,18.304,18.175,15.469,13.379,13.245
1,Armenia,ARM,1869848,4622,13696067548,2021-05-16,212,13,8522,<NA>,...,17.498,18.261,17.617,17.704,18.966,18.304,18.175,15.469,13.379,13.245
2,Armenia,ARM,1869848,4622,13696067548,2021-05-23,157,6,7313,<NA>,...,17.498,18.261,17.617,17.704,18.966,18.304,18.175,15.469,13.379,13.245
3,Armenia,ARM,1869848,4622,13696067548,2021-05-30,108,7,6967,<NA>,...,17.498,18.261,17.617,17.704,18.966,18.304,18.175,15.469,13.379,13.245
4,Armenia,ARM,1869848,4622,13696067548,2021-06-06,73,0,12974,<NA>,...,17.498,18.261,17.617,17.704,18.966,18.304,18.175,15.469,13.379,13.245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Turkey,TUR,63097818,9042,741479101044,2022-05-27,940,4,873,1150,...,9.901,10.304,10.899,10.919,10.956,13.730,13.148,11.969,10.465,9.388
31436,Turkey,TUR,63097818,9042,741479101044,2022-05-28,966,3,304,479,...,9.901,10.304,10.899,10.919,10.956,13.730,13.148,11.969,10.465,9.388
31437,Turkey,TUR,63097818,9042,741479101044,2022-05-29,864,2,144,279,...,9.901,10.304,10.899,10.919,10.956,13.730,13.148,11.969,10.465,9.388
31438,Turkey,TUR,63097818,9042,741479101044,2022-05-30,908,4,973,1489,...,9.901,10.304,10.899,10.919,10.956,13.730,13.148,11.969,10.465,9.388


### Dane o przestępstwach w latach

In [111]:
df_crime = pd.read_csv("load_data\\crime_rate_per_100k.csv")
df_crime = df_crime.iloc[:217]
df_crime.drop(columns=["Series Name", "Series Code", "Country Name"], inplace=True, errors="ignore")
df_crime.columns = df_crime.columns.str.replace(r"\s*\[YR\d+\]", "", regex=True)
df_crime = df_crime.melt(id_vars=["Country Code"], var_name="Year", value_name="Crime Rate")
df_crime["Year"] = df_crime["Year"].astype(int)
df_crime["Crime Rate"] = pd.to_numeric(df_crime["Crime Rate"], errors="coerce")
df_crime.dropna(subset=["Crime Rate"], inplace=True)
df_crime = df_crime.pivot(index="Country Code", columns="Year", values="Crime Rate")
df_crime.columns = [f"Murders_Rate_{int(year)}" for year in df_crime.columns]
df_crime.reset_index(inplace=True)
df_final = df_unemployment.merge(df_crime, left_on="3_letter_code", right_on="Country Code", how="left")
df_final.drop(columns=["Country Code"], inplace=True)
df_final.to_csv("..\wyniki\\final_data.csv",  index=False)
display(df_final)

<>:14: SyntaxWarning: invalid escape sequence '\w'
<>:14: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_12068\4286031475.py:14: SyntaxWarning: invalid escape sequence '\w'
  df_final.to_csv("..\wyniki\\final_data.csv",  index=False)


,Country_name,3_letter_code,population_urban,gdp_per_capita_usd,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vaccinations,New_persons_fully_vaccinated,...,Murders_Rate_2012,Murders_Rate_2013,Murders_Rate_2014,Murders_Rate_2015,Murders_Rate_2016,Murders_Rate_2017,Murders_Rate_2018,Murders_Rate_2019,Murders_Rate_2020,Murders_Rate_2021
0,Armenia,ARM,1869848,4622,13696067548,2021-05-09,243,7,10162,<NA>,...,2.230289,2.171377,2.49141,2.605438,3.035764,2.454484,1.76270,2.588100,1.853430,2.185617
1,Armenia,ARM,1869848,4622,13696067548,2021-05-16,212,13,8522,<NA>,...,2.230289,2.171377,2.49141,2.605438,3.035764,2.454484,1.76270,2.588100,1.853430,2.185617
2,Armenia,ARM,1869848,4622,13696067548,2021-05-23,157,6,7313,<NA>,...,2.230289,2.171377,2.49141,2.605438,3.035764,2.454484,1.76270,2.588100,1.853430,2.185617
3,Armenia,ARM,1869848,4622,13696067548,2021-05-30,108,7,6967,<NA>,...,2.230289,2.171377,2.49141,2.605438,3.035764,2.454484,1.76270,2.588100,1.853430,2.185617
4,Armenia,ARM,1869848,4622,13696067548,2021-06-06,73,0,12974,<NA>,...,2.230289,2.171377,2.49141,2.605438,3.035764,2.454484,1.76270,2.588100,1.853430,2.185617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Turkey,TUR,63097818,9042,741479101044,2022-05-27,940,4,873,1150,...,4.272196,NaN,NaN,2.806161,3.337472,3.095390,2.60961,2.419692,2.505484,2.519599
31436,Turkey,TUR,63097818,9042,741479101044,2022-05-28,966,3,304,479,...,4.272196,NaN,NaN,2.806161,3.337472,3.095390,2.60961,2.419692,2.505484,2.519599
31437,Turkey,TUR,63097818,9042,741479101044,2022-05-29,864,2,144,279,...,4.272196,NaN,NaN,2.806161,3.337472,3.095390,2.60961,2.419692,2.505484,2.519599
31438,Turkey,TUR,63097818,9042,741479101044,2022-05-30,908,4,973,1489,...,4.272196,NaN,NaN,2.806161,3.337472,3.095390,2.60961,2.419692,2.505484,2.519599
